# Double pendulum. Hamiltonian dynamics

In [ ]:
using DiffEqPhysics
using DifferentialEquations
using Plots

### Hamiltonian $\mathcal{H}(p, \theta)$

$$
\mathcal{H} = \dfrac{1}{2}
\dfrac{p_1^2 + 2 p_2^2 - 2 p_1 p_2 \cos(\theta_1 - \theta_2)}{2 - \cos(\theta_1 - \theta_2)^2} +
3 - 2 \cos{\theta_1} - \cos{\theta_2}
$$

In [ ]:
function H(p, q, param = nothing)
    E = (1/2)*(p[1]^2 + 2*p[2]^2 - 2*p[1]*p[2]*cos(q[1] - q[2]))/(2 - cos(q[1] - q[2])^2) + 3 - 2*cos(q[1]) - cos(q[2])
end

### Time span and initial conditions

In [ ]:
t_max = 384.0
t_step = 0.02

p0 = [0.0, 0.0]
q0 = [0.0, pi/2]
#q0 = [3.0*pi/12, -acos(1.0 - 2*cos(3.0*pi/12))]
H(p0, q0)

### Numerical solution

In [ ]:
problem = HamiltonianProblem(H, p0, q0, (0, t_max))
solution = solve(problem, Vern6(), adaptive=false, dt = t_step)
data = reduce(hcat, solution.u)'

### Orbit $(\theta_1, p_1)$

In [ ]:
Plots.scalefontsizes()
Plots.scalefontsizes(0.5)
plotOrbit = scatter(
    data[:, 3], data[:, 1],
    aspect_ratio = 1,
    xlabel = "\\theta_1",
    ylabel = "\\pi_1",
    plot_title = "Double pendulum",
    markersize = 0.4,
    alpha = 0.15,
    legend = false,
    dpi = 300
)
annotate!(
    xlims(plotOrbit)[1] + 0.12*(xlims(plotOrbit)[2] - xlims(plotOrbit)[1]),
    ylims(plotOrbit)[2] - 0.03*(ylims(plotOrbit)[2] - ylims(plotOrbit)[1]),
    text("E = $(round(H(p0, q0), digits=4))", :red, :right, 4)
)

### Integration error

In [ ]:
Plots.scalefontsizes()
Plots.scalefontsizes(0.5)
plotEnergy = scatter(
    solution.t,
    map(eachrow(data)) do s
        H([s[1], s[2]], [s[3], s[4]]) - H(p0, q0)
    end,
    xlabel = "t",
    ylabel = "Energy error - \\Delta E",
    plot_title = "Double pendulum",
    markersize = 0.4,
    alpha = 0.15,
    legend = false,
    dpi = 300
)